In [1]:
from ROOT import TFile, TTree, TChain
from ROOT import gROOT, AddressOf
import uproot
from array import array
from tqdm import tqdm
import numpy as np
from helpers import samples, timeBinChoice

## 0 through 6
TimeChoice = 3
good_times_ano, good_times_cat = timeBinChoice(TimeChoice)
print("TimeBinChoice Anode", good_times_ano, "Cathode", good_times_cat)
box = False
#Setting box=True will only consider signal events that decay right in front of a chamber to be in acceptance

Welcome to JupyROOT 6.18/04
('TimeBinChoice Anode', [7, 8, 9], 'Cathode', [6, 7, 8])


In [2]:
def processSingleSample(dataset):

    print("Processing dataset", dataset)

    comp_ME11 = array('f', [0.])
    comp_ME12 = array('f', [0.])
    comp_ME13 = array('f', [0.])
    comp_ME21 = array('f', [0.])
    comp_ME22 = array('f', [0.])
    comp_ME31 = array('f', [0.])
    comp_ME32 = array('f', [0.])
    comp_ME41 = array('f', [0.])
    comp_ME42 = array('f', [0.])

    wire_ME11 = array('f', [0.])
    wire_ME12 = array('f', [0.])
    wire_ME13 = array('f', [0.])
    wire_ME21 = array('f', [0.])
    wire_ME22 = array('f', [0.])
    wire_ME31 = array('f', [0.])
    wire_ME32 = array('f', [0.])
    wire_ME41 = array('f', [0.])
    wire_ME42 = array('f', [0.])

    if box:
        out_path = "trees_TimeChoice" + str(TimeChoice) + "/box_selected/MaxDigis_tree_" + dataset + ".root"
    else:
        out_path = "trees_TimeChoice" + str(TimeChoice) + "/MaxDigis_tree_" + dataset + ".root"
        
    f = TFile.Open( out_path, 'RECREATE' )
    if not f:
        print("Failed to open output file:", f)

    comp_tree = TTree( 'comparator', 'Max Comparator digis in a single chamber in each ring per event' )
    wire_tree = TTree( 'wire', 'Max Comparator digis in a single chamber in each ring per event' )
    # tree.Branch( 'digis', digis, 'Ev_max_nComp_ME11/I:Ev_max_nComp_ME12:Ev_max_nComp_ME13:Ev_max_nComp_ME21:Ev_max_nComp_ME22:Ev_max_nComp_ME31:Ev_max_nComp_ME32:Ev_max_nComp_ME41:Ev_max_nComp_ME42:Ev_max_nWire_ME11:Ev_max_nWire_ME12:Ev_max_nWire_ME13:Ev_max_nWire_ME21:Ev_max_nWire_ME22:Ev_max_nWire_ME31:Ev_max_nWire_ME32:Ev_max_nWire_ME41:Ev_max_nWire_ME42',0 )
    comp_tree.Branch( 'Ev_max_nComp_ME11', comp_ME11, 'comp_ME11/F')
    comp_tree.Branch( 'Ev_max_nComp_ME12', comp_ME12, 'comp_ME12/F')
    comp_tree.Branch( 'Ev_max_nComp_ME13', comp_ME13, 'comp_ME13/F')
    comp_tree.Branch( 'Ev_max_nComp_ME21', comp_ME21, 'comp_ME21/F')
    comp_tree.Branch( 'Ev_max_nComp_ME22', comp_ME22, 'comp_ME22/F')
    comp_tree.Branch( 'Ev_max_nComp_ME31', comp_ME31, 'comp_ME31/F')
    comp_tree.Branch( 'Ev_max_nComp_ME32', comp_ME32, 'comp_ME32/F')
    comp_tree.Branch( 'Ev_max_nComp_ME41', comp_ME41, 'comp_ME41/F')
    comp_tree.Branch( 'Ev_max_nComp_ME42', comp_ME42, 'comp_ME42/F')

    wire_tree.Branch( 'Ev_max_nWire_ME11', wire_ME11, 'wire_ME11/F')
    wire_tree.Branch( 'Ev_max_nWire_ME12', wire_ME12, 'wire_ME12/F')
    wire_tree.Branch( 'Ev_max_nWire_ME13', wire_ME13, 'wire_ME13/F')
    wire_tree.Branch( 'Ev_max_nWire_ME21', wire_ME21, 'wire_ME21/F')
    wire_tree.Branch( 'Ev_max_nWire_ME22', wire_ME22, 'wire_ME22/F')
    wire_tree.Branch( 'Ev_max_nWire_ME31', wire_ME31, 'wire_ME31/F')
    wire_tree.Branch( 'Ev_max_nWire_ME32', wire_ME32, 'wire_ME32/F')
    wire_tree.Branch( 'Ev_max_nWire_ME41', wire_ME41, 'wire_ME41/F')
    wire_tree.Branch( 'Ev_max_nWire_ME42', wire_ME42, 'wire_ME42/F')

    if dataset=="Data_pre-selected":
        comp_pass_list = open("passlists/comp_pass_list.txt","w")
        wire_pass_list = open("passlists/wire_pass_list.txt","w")
        comp_last_text = ""
        wire_last_text = ""
        compXX = [98, 56, 30, 49, 42, 49, 35, 42, 31]
        wireXX = [104, 92, 32, 133, 83, 130, 74, 127, 88]
        comp_pass_count = 0
        wire_pass_count = 0
        
    if dataset=="Data" or dataset=="Data_pre-selected":
      MC = False
      Neutrino = False
    elif dataset=="Neutrino_gun" or dataset=="Nu_E10-pythia8-gun":
      MC = False
      Neutrino = True
    else:
      MC = True
      Neutrino = False

    print("Declaring TChains")

    ## declare the chains
    alctChain = TChain("lctreader/Ev_alcttree")
    clctChain = TChain("lctreader/Ev_clcttree")
    llpChain = TChain("lctreader/llp")

    prefix = "root://cmseos.fnal.gov/"
    userdir = "/store/user/dildick/"
    directory = prefix + userdir + samples[dataset]

    print("Adding Files")

    llpChain.Add(directory + "TPEHists_*.root")
    alctChain.Add(directory + "TPEHists_*.root")
    clctChain.Add(directory + "TPEHists_*.root")

    print("Entries in ALCT Chain", alctChain.GetEntries())
    print("Entries in CLCT Chain", clctChain.GetEntries())
    print("Entries in LLP Chain", llpChain.GetEntries())

    clct = clctChain
    alct = alctChain
    llp = llpChain

    nEntries_clct = clctChain.GetEntries()
    nEntries_alct = alctChain.GetEntries()
    if MC:
      nEntries_llp = llpChain.GetEntries()
              
    llp_accept = []
    #max_nComp = np.array([0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0])
    max_nComp = np.zeros((5,4))
    max_nWire = np.zeros((5,4))
    lastEventclct = 0
    lastEventalct = 0
    n_big = 0
              
    passed=0
    if MC:
        print "Finding LLP Acceptance"
        for i in range(0, nEntries_llp):
            llp.GetEntry(i) 

            if(llp.llp_in_acceptance[0]==1 or llp.llp_in_acceptance[1]==1):
                if box:
                    for nlp in range(0,2):
                        R = get_R((llp.llp_decay_x[nlp]/100),(llp.llp_decay_y[nlp]/100))
                        for cut in range(0,len(zMin)):
                            if(abs(llp.llp_decay_z[nlp]/100) > zMin[cut] and abs(llp.llp_decay_z[nlp]/100) < zMax[cut] and R > rMin[cut] and R < rMax[cut]):
                                llp_accept.append(llp.event)
                                passed+=1
                                break
                else:
                    llp_accept.append(llp.event)
                    passed+=1

    print "# passed = " + str(passed)
              
              
                            
    print "Starting CLCT Analysis"
    for i in range(0, nEntries_clct):
        clct.GetEntry(i)

        if MC:
            if clct.t_Event not in llp_accept:
                continue

        in_time = False
        nComp = 0
        for compy in range(len(clct.t_compTimes)):
            for bx in range(len(clct.t_compTimes[compy])):
                if clct.t_compTimes[compy][bx] in good_times_cat:
                    in_time = True
                    nComp += 1
        if not in_time: continue

        if(clct.t_Event!=lastEventclct):
            lastEventclct = clct.t_Event

            comp_ME11[0] = max_nComp[1][1]
            comp_ME12[0] = max_nComp[1][2]
            comp_ME13[0] = max_nComp[1][3]
            comp_ME21[0] = max_nComp[2][1]
            comp_ME22[0] = max_nComp[2][2]
            comp_ME31[0] = max_nComp[3][1]
            comp_ME32[0] = max_nComp[3][2]
            comp_ME41[0] = max_nComp[4][1]
            comp_ME42[0] = max_nComp[4][2]
            max_nComp = np.zeros((5,4))

            comp_tree.Fill()


        if nComp > max_nComp[clct.t_station][clct.t_ring]:
            max_nComp[clct.t_station][clct.t_ring] = nComp

        if dataset=="Data_pre-selected":
            if (max_nComp[1][1]>compXX[0] or max_nComp[1][2]>compXX[1] or max_nComp[1][3]>compXX[2] or
                max_nComp[2][1]>compXX[3] or max_nComp[2][2]>compXX[4] or
                max_nComp[3][1]>compXX[5] or max_nComp[3][2]>compXX[6] or
                max_nComp[4][1]>compXX[7] or max_nComp[4][2]>compXX[8]):

                comp_pass_text = "%i:%i:%d\n" % (clct.t_Run,clct.t_Lumi,clct.t_Event)
                if comp_pass_text!=comp_last_text:
                    comp_pass_list.write(comp_pass_text)
                    comp_pass_count+=1
                comp_last_text = comp_pass_text

    if dataset=="Data_pre-selected":
        print "%i events passed cut" %(comp_pass_count)

              
    print "Starting ALCT Analysis"
    for i in range(0, nEntries_alct):
        alct.GetEntry(i)

        if MC:
            if alct.t_Event not in llp_accept:
                continue

        in_time = False
        nWire = 0
        for wirey in range(len(alct.t_wireTimes)):
            for bx in range(len(alct.t_wireTimes[wirey])):
                if alct.t_wireTimes[wirey][bx] in good_times_ano:
                    in_time = True
                    nWire += 1
        if not in_time: continue

        if(alct.t_Event!=lastEventalct):
            lastEventalct = alct.t_Event

            wire_ME11[0] = max_nWire[1][1]
            wire_ME12[0] = max_nWire[1][2]
            wire_ME13[0] = max_nWire[1][3]
            wire_ME21[0] = max_nWire[2][1]
            wire_ME22[0] = max_nWire[2][2]
            wire_ME31[0] = max_nWire[3][1]
            wire_ME32[0] = max_nWire[3][2]
            wire_ME41[0] = max_nWire[4][1]
            wire_ME42[0] = max_nWire[4][2]
            max_nWire = np.zeros((5,4))

            wire_tree.Fill()


        if nWire > max_nWire[alct.t_station][alct.t_ring]:
            max_nWire[alct.t_station][alct.t_ring] = nWire

        if dataset=="Data_pre-selected":
            if (max_nWire[1][1]>wireXX[0] or max_nWire[1][2]>wireXX[1] or max_nWire[1][3]>wireXX[2] or
                max_nWire[2][1]>wireXX[3] or max_nWire[2][2]>wireXX[4] or
                max_nWire[3][1]>wireXX[5] or max_nWire[3][2]>wireXX[6] or
                max_nWire[4][1]>wireXX[7] or max_nWire[4][2]>wireXX[8]):

                wire_pass_text = "%i:%i:%d\n" % (alct.t_Run,alct.t_Lumi,alct.t_Event)
                if wire_pass_text!=wire_last_text:
                    wire_pass_list.write(wire_pass_text)
                    wire_pass_count+=1
                wire_last_text = wire_pass_text

    if dataset=="Data_pre-selected":
        print "%i events passed cut" %(wire_pass_count)
              
              
              
    f.Write()
    f.Close()
    if dataset=="Data_pre-selected":
        comp_pass_list.close()
        wire_pass_list.close()
    print("Finished analysis")              

In [ ]:
print("List all datasets:")
for dataset in tqdm(samples):    
    print(dataset)      
    
for dataset in tqdm(samples):    
    processSingleSample(dataset)          

  0%|          | 0/30 [00:00<?, ?it/s]

List all datasets:
HTo2LongLivedTo4b_MH_125_MFF_12_CTau_9000mm
HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm
HTo2LongLivedTo4q_MH_125_MFF_1_CTau_1000mm
HTo2LongLivedTo4b_MH_250_MFF_60_CTau_10000mm
HTo2LongLivedTo4b_MH_1000_MFF_450_CTau_100000mm
HTo2LongLivedTo4b_MH_350_MFF_160_CTau_10000mm
HTo2LongLivedTo4b_MH_250_MFF_120_CTau_1000mm
HTo2LongLivedTo4b_MH_125_MFF_50_CTau_30000mm
HTo2LongLivedTo4b_MH_350_MFF_160_CTau_5000mm
HTo2LongLivedTo4q_MH_125_MFF_1_CTau_10000mm
HTo2LongLivedTo4b_MH_350_MFF_160_CTau_1000mm
HTo2LongLivedTo4b_MH_125_MFF_50_CTau_3000mm
HTo2LongLivedTo4b_MH_250_MFF_60_CTau_1000mm
HTo2LongLivedTo4b_MH_350_MFF_80_CTau_1000mm
ZeroBias_2018D_0001
ZeroBias_2018D_0000
ZeroBias_2018D_0003
ZeroBias_2018D_0002
ZeroBias_2018D_0005
ZeroBias_2018D_0004
ZeroBias_2018D_0007
ZeroBias_2018D_0006
HTo2LongLivedTo4b_MH_125_MFF_25_CTau_15000mm
HTo2LongLivedTo4b_MH_350_MFF_80_CTau_500mm
HTo2LongLivedTo4b_MH_350_MFF_80_CTau_10000mm
HTo2LongLivedTo4b_MH_125_MFF_12_CTau_900mm
HTo2LongLivedTo4b_MH

  3%|▎         | 1/30 [01:25<41:29, 85.83s/it]

Finished analysis
('Processing dataset', 'HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm')
Declaring TChains
Adding Files
('Entries in ALCT Chain', 41828L)
('Entries in CLCT Chain', 39165L)
('Entries in LLP Chain', 48500L)
Finding LLP Acceptance
# passed = 185
Starting CLCT Analysis
Starting ALCT Analysis


  7%|▋         | 2/30 [01:59<32:49, 70.33s/it]

Finished analysis
('Processing dataset', 'HTo2LongLivedTo4q_MH_125_MFF_1_CTau_1000mm')
Declaring TChains
Adding Files
('Entries in ALCT Chain', 0L)
('Entries in CLCT Chain', 0L)
('Entries in LLP Chain', 0L)

 10%|█         | 3/30 [02:05<22:50, 50.76s/it]


Finding LLP Acceptance
# passed = 0
Starting CLCT Analysis
Starting ALCT Analysis
Finished analysis
('Processing dataset', 'HTo2LongLivedTo4b_MH_250_MFF_60_CTau_10000mm')
Declaring TChains
Adding Files
('Entries in ALCT Chain', 544865L)
('Entries in CLCT Chain', 345162L)
('Entries in LLP Chain', 40000L)
Finding LLP Acceptance
# passed = 4138
Starting CLCT Analysis
